In [ ]:
# Cell 1: Environment Setup and Configuration (Model C: Pure RL)
# Sets up the computational environment for the Pure Reinforcement Learning model.
# Note: Unlike Model B, this configuration excludes the fixed Strongly Entangling Layers (SEL),
# focusing solely on the adaptive circuit structure generated by the RL agent.
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
import random
import torch

# Check for CUDA availability to enable GPU acceleration
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))
    print("Torch CUDA version:", torch.version.cuda)

In [ ]:
# Cell 2: Quantum Device Initialization
# Configures the quantum simulator. 'lightning.qubit' is a high-performance C++ backend.
# This device will execute the variational circuits constructed by the RL agent.

dev = qml.device("lightning.qubit", wires=6, shots=None)

In [ ]:
# Cell 3: Parameter Counting Utility
# Calculates the number of trainable parameters required for the dynamically generated circuit.
# Iterates through the gate sequence to count parameterized gates (e.g., 'Rot').
# Note: In Model C (Pure RL), this count represents the total number of trainable parameters in the circuit,
# as there are no pre-defined Strongly Entangling Layers.

def check_np(list):
    """
    Counts the number of parameterized gates in the generated circuit architecture.
    Args:
        list: The list of gates (operations) defining the circuit.
    Returns:
        int: The total count of gates requiring optimization parameters.
    """
    num = 0
    for r in list:
        # Check if the gate is a Rotation gate ('Rot') which requires parameters.
        if r[0] == "Rot": 
            num += 1
    return num

In [ ]:
# Cell 4: Dynamic Ansatz Construction (RL-driven)
# Constructs the variational circuit based solely on the gate sequence (gatestream) provided by the RL agent.
# In this Pure RL model (Model C), this function defines the entire trainable unitary evolution.

def ansatz(W, gatestream):
    """
    Builds the quantum circuit dynamically.
    Args:
        W: Trainable parameters for rotation gates.
        gatestream: List of gates (Action history from RL agent).
    """
    w_cnt = 0
    for gate in gatestream:
        # If the gate is a parameterized rotation
        if gate[0] == "Rot":
            # Apply Pauli Rotation: Param W[w_cnt], Axis gate[1], Target wire gate[2]
            qml.PauliRot(W[w_cnt], gate[1], wires=gate[2])
            w_cnt += 1

        # If the gate is CNOT (Entanglement)
        elif gate[0] == "CNOT":
            qml.CNOT(wires=[gate[1], gate[2]])

In [ ]:
# Cell 5: State Preparation (Amplitude Embedding)
# Maps classical input vectors into the quantum Hilbert space.

def statepreparation(x):
    """
    Encodes the input feature vector x into quantum state amplitudes.
    Args:
        x (tensor): Normalized input feature vector.
    """
    # Amplitude Embedding: Encodes N features into log2(N) qubits.
    # Normalization ensures the input vector represents a valid quantum state.
    qml.AmplitudeEmbedding(x, wires=range(6), normalize=True)

In [ ]:
# Cell 6: Quantum Node (QNode) Definition
# Defines the quantum circuit execution pipeline using the 'adjoint' differentiation method.
# In Model C (Pure RL), the circuit architecture is entirely determined by the RL agent (gatestream),
# without any pre-fixed Strongly Entangling Layers.

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def circuit(weights_rl, x, gatestream):
    """
    Executes the variational quantum circuit.
    Args:
        weights_rl (tensor): Trainable parameters for the RL-generated rotation gates.
        x (tensor): Input feature vector.
        gatestream (list): The sequence of gates defined by the RL agent.
    Returns:
        float: Expectation value of the Pauli-Z operator on the first qubit.
    """
    
    # 1. State Preparation (Data Embedding)
    statepreparation(x)
    
    # 2. Variational Circuit (RL-driven Ansatz)
    # Applies the dynamic circuit structure optimized by the RL agent.
    # Note: Unlike Model B, 'weights_ent' is not used, and no fixed entanglement layers are applied.
    ansatz(weights_rl, gatestream)

    # 3. Measurement
    # Computes the expectation value <Z> as the model prediction (Range: [-1, 1]).
    return qml.expval(qml.PauliZ(0))

In [ ]:
# Cell 7: Variational Quantum Classifier (VQC) Model
# Combines the quantum circuit output with a classical bias term to form the final prediction model.

def variational_classifier(weights_rl, bias, x, gatestream):
    """
    Computes the classifier output for a given input x.
    Args:
        weights_rl (tensor): Trainable parameters for the RL-generated circuit.
        bias (tensor): Classical bias term to shift the decision boundary.
        x (tensor): Input data vector.
        gatestream (list): Circuit architecture defined by the RL agent.
    Returns:
        tensor: The raw prediction score (logit) before thresholding.
    """
    # The model output is the expectation value of the quantum circuit plus a classical bias.
    # y_pred = <Z> + b
    return circuit(weights_rl, x, gatestream) + bias

In [ ]:
# Cell 8: Mean Squared Error (MSE) Loss
# Standard loss function for regression-based quantum classification.

def square_loss(labels, predictions):
    """
    Computes the Mean Squared Error between labels and predictions.
    Includes tensor type/device compatibility checks.
    """
    # Ensure predictions are a tensor
    preds  = torch.stack(predictions).squeeze() if isinstance(predictions, list) else predictions
    # Match label tensor properties to predictions
    labels = torch.as_tensor(labels, dtype=preds.dtype, device=preds.device).squeeze()
    
    # Calculate Mean Squared Error
    return ((labels - preds) ** 2).mean()

In [ ]:
# Cell 9: Classification Accuracy (Sign-based)
# Measures accuracy by comparing the sign of predictions with true labels {-1, 1}.

def accuracy_sign(labels, predictions):
    """
    Computes accuracy for binary classification.
    Method: Sign thresholding (pred >= 0 -> Class 1, else -> Class -1).
    Args:
        labels: True labels tensor.
        predictions: Model output tensor (expectation values).
    """
    # Ensure tensor compatibility
    preds  = torch.stack(predictions).squeeze() if isinstance(predictions, list) else predictions
    labels = labels.to(dtype=preds.dtype, device=preds.device).squeeze()
    
    # Convert continuous predictions to discrete class labels {1.0, -1.0}
    pred_labels = torch.where(preds >= 0, 1.0, -1.0)
    
    # Calculate mean accuracy
    return (pred_labels == labels).float().mean()

In [ ]:
# Cell 10: Cost Function Calculation
# Computes the total cost (loss) over a batch of data by aggregating individual predictions.
# In Model C, the optimization is performed solely on the RL-generated circuit parameters (weights_rl).

def cost(weights_rl, bias, X, Y, gatestream):
    """
    Calculates the loss for a given dataset batch.
    Args:
        weights_rl (tensor): Trainable parameters for the RL-generated circuit.
        bias (tensor): Classical bias term.
        X (tensor): Batch of input data.
        Y (tensor): Batch of true labels.
        gatestream (list): Circuit structure defined by the RL agent.
    Returns:
        tensor: Scalar loss value used for gradient descent.
    """
    # Generate predictions for each sample in the batch
    # Note: 'variational_classifier' is called without 'weights_ent' in this Pure RL model.
    predictions = [variational_classifier(weights_rl, bias, x, gatestream) for x in X]
    
    # Calculate Square Loss (MSE) between predictions and ground truth
    return square_loss(Y, predictions)

In [ ]:
# Cell 11: Optimization Routine (Local Training)
# Trains the parameters of the specific circuit architecture generated by the RL agent.
# This function acts as the "inner loop" of the reinforcement learning process,
# providing the reward signal (accuracy) based on the circuit's performance.
# Note: In Model C, optimization is applied solely to the RL-generated parameters (weights_rl) and bias.

def opt_classifier(gatestream, iters=15, draw=False):
    """
    Optimizes the parameters of the quantum circuit for a fixed structure.
    Args:
        gatestream (list): The circuit architecture defined by the RL agent.
        iters (int): Number of optimization iterations.
        draw (bool): Whether to generate a circuit diagram after training.
    Returns:
        tuple: Optimization logs, circuit diagram string, and final model state.
    """
    import os
    
    # 1. Data Loading and Preprocessing
    # Load the Speck dataset (features and labels).
    X = torch.from_numpy(np.load(os.path.join("dataset", "data_speck.npy"))).float()
    Y = torch.from_numpy(np.load(os.path.join("dataset", "labels_speck.npy"))).float()
    Y = Y * 2 - 1 # Rescale labels from [0, 1] to [-1, 1] for hinge-like loss compatibility.

    # 2. Data Splitting
    np.random.seed(0) # Fix seed for reproducibility
    
    num_data = len(Y)
    num_train = int(0.8 * num_data) # 80% Training, 20% Validation
    indices = torch.randperm(num_data, device=Y.device) 
    X_train = X[indices[:num_train]]
    Y_train = Y[indices[:num_train]]
    X_val   = X[indices[num_train:]]
    Y_val   = Y[indices[num_train:]]

    dev_t = X.device

    # 3. Parameter Initialization
    # Count the number of required parameters for the RL-generated circuit.
    num_rot_params = check_np(gatestream)
    
    if num_rot_params > 0:
        # Initialize weights with small random values to break symmetry.
        weights_rl = (torch.randn(num_rot_params, device=dev_t) * 0.01).requires_grad_(True)
    else:
        # Handle case where no parameterized gates exist in the circuit.
        weights_rl = torch.zeros(0, device=dev_t, requires_grad=True)

    # Initialize classical bias term
    bias = torch.tensor(0.0, device=dev_t, requires_grad=True)

    # Define Optimizer: Stochastic Gradient Descent (SGD) with Momentum.
    # Note: Only 'weights_rl' and 'bias' are optimized.
    opt = torch.optim.SGD([weights_rl, bias], lr=0.01, momentum=0.9)
    batch_size = 10

    out_list = []

    # 4. Optimization Loop
    for it in range(iters):
        
        # Mini-batch sampling
        batch_idx = torch.randint(0, num_train, (batch_size,), device=dev_t)
        X_batch, Y_batch = X_train[batch_idx], Y_train[batch_idx]

        # Backpropagation step
        opt.zero_grad()
        # Compute loss (Model C: cost function does not take weights_ent)
        loss = cost(weights_rl, bias, X_batch, Y_batch, gatestream)
        loss.backward()
        opt.step()

        # 5. Evaluation and Logging
        # Compute performance metrics without updating gradients.
        with torch.no_grad():
            
            # Training Metrics
            vals_tr  = [variational_classifier(weights_rl, bias, x, gatestream) for x in X_train]
            preds_tr = torch.stack(vals_tr).squeeze()
            acc_train = (torch.sign(preds_tr) == torch.sign(Y_train)).float().mean()

            # Validation Metrics
            vals_val  = [variational_classifier(weights_rl, bias, x, gatestream) for x in X_val]
            preds_val = torch.stack(vals_val).squeeze()
            acc_val   = (torch.sign(preds_val) == torch.sign(Y_val)).float().mean()
            cost_val  = ((Y_val - preds_val) ** 2).mean()

        out_list.append([it + 1, float(cost_val), float(acc_train), float(acc_val)])

        # Early Stopping: Terminate if validation accuracy reaches the threshold (0.65).
        if float(acc_val) >= 0.65:
            break

    # 6. Circuit Visualization
    if draw and len(X_val) > 0:
        x_draw     = X_val[0].detach().cpu().numpy()
        w_rl_draw  = weights_rl.detach().cpu().numpy()
        # Draw the circuit structure using PennyLane's drawer
        draw_p = qml.draw(circuit)(w_rl_draw, x_draw, gatestream)
    else:
        draw_p = None    

    # Bundle final state for return
    figset = [weights_rl.detach(), bias.detach(), X_train, Y_train, X_val, Y_val]
    
    return out_list, draw_p, figset

In [ ]:
# Cell 12: Gate Encoding Utility (Symbolic to Numerical)
# Converts discrete gate definitions into numerical vectors for the RL agent's state representation.
# This encoding facilitates the neural network's processing of circuit structures.

def gate_to_obs(gate):
    """
    Transforms a gate specification into a fixed-size numerical observation vector.
    Encoding Schema: [Is_Rot, Is_CNOT, Parameter_1 (Axis/Control), Parameter_2 (Target)]
    Args:
        gate (list): Symbolic gate representation (e.g., ['Rot', 'X', 0]).
    Returns:
        list: Numerical vector representation (e.g., [1, 0, 1, 0]).
    """
    
    # Initialize observation vector [Type_Rot, Type_CNOT, Param1, Param2]
    ob = [0, 0, 0, 0]
    
    if gate[0] == 'Rot':
        ob[0] = 1 # Set Rotation Flag
        
        # Encode Rotation Axis: X=1, Y=2, Z=3
        if gate[1] == 'X': ob[2] = 1
        elif gate[1] == 'Y': ob[2] = 2
        elif gate[1] == 'Z': ob[2] = 3
        
        ob[3] = gate[2] # Target Qubit Index
    
    elif gate[0] == 'CNOT':
        ob[1] = 1 # Set CNOT Flag
        ob[2] = gate[1] # Control Qubit Index
        ob[3] = gate[2] # Target Qubit Index
    
    return ob

In [ ]:
# Cell 13: State Update Mechanism
# Updates the environment's internal state and observation vector based on the agent's selected action.
# This function physically adds the chosen gate to the circuit architecture.

def update_obs(act, step, obs, gatestream, gates):
    """
    Applies the selected action to the environment.
    Args:
        act (int): Index of the selected action (gate).
        step (int): Current time step (depth of the circuit).
        obs (list): Current observation vector (history of gates).
        gatestream (list): Current sequence of gates defining the circuit.
        gates (list): List of all possible gates (Action Space).
    Returns:
        tuple: Updated step count, observation vector, and gatestream.
    """
    
    # 1. Circuit Construction
    # Append the selected gate to the growing circuit sequence.
    gatestream.append(gates[act])
    
    # 2. Observation Update
    # Convert the symbolic gate to its numerical representation.
    ob = gate_to_obs(gates[act])
    # Update the observation vector at the current time step.
    obs[step] = ob
    
    # 3. Increment Time Step
    step += 1

    return step, obs, gatestream

In [ ]:
# Cell 14: Reward Function Definition (Multi-objective)
# Calculates the scalar reward signal for the RL agent based on circuit performance and structural heuristics.
# The reward acts as the fitness function, guiding the agent towards circuits that are both
# accurate (high classification performance) and efficient (low depth, uniform gate distribution).

def cal_reward(steps, obs, outs):
    """
    Computes the reward for the current step.
    Args:
        steps (int): Current depth of the circuit (number of gates added).
        obs (list): History of gates (observation vectors).
        outs (list): Optimization logs from 'opt_classifier' [[iter, cost, acc_train, acc_val], ...].
    Returns:
        tuple: (List of individual reward components, Weighted Total Scalar Reward)
    """
    
    ## 1. Performance-based Rewards
    # Accuracy: Average training accuracy over the optimization trajectory.
    acc = [row[2] for row in outs]
    acc_m = sum(acc) / len(acc)

    # Cost: Inverse of the average loss (Lower loss -> Higher reward).
    cost = [row[1] for row in outs]
    cost_m = 1 / (sum(cost) / len(cost))

    ## 2. Structural Uniformity Reward (Variance)
    # Encourages the agent to distribute gates evenly across all qubits, preventing bottlenecks.
    pop_list = [0, 0, 0, 0, 0, 0] # Counter for gate usage per qubit
    for row in obs:
        if row[1] == 1: # CNOT gate involves two qubits
            pop_list[row[2]] += 1
            pop_list[row[3]] += 1
        elif row[0] == 1: # Rotation gate involves one qubit
            pop_list[row[3]] += 1
    # Reward is higher if variance of gate counts is lower.
    pop_r = (2 - np.var(pop_list)) / 2    

    ## 3. Redundancy Penalty (Duplicate Check)
    # Penalizes the agent for applying the exact same gate consecutively on the same qubits.
    # This prevents useless operations like Rot(x) followed immediately by Rot(x).
    dup_r = 0
    if obs[steps-1][0] == 1: # Check Rotation duplicates
        tc = obs[steps-1][3]
        tc_list = []
        for row in obs:
            if row[1] == 1:
                if row[2] == tc or row[3] == tc: tc_list.append(row)
            elif row[0] == 1:
                if row[3] == tc: tc_list.append(row)
        if len(tc_list) > 1:
            if tc_list[-1] == tc_list[-2]: dup_r = -10 # Heavy penalty for duplicates
            
    elif obs[steps-1][1] == 1: # Check CNOT duplicates
        # Verify both Control and Target qubit histories
        tc = obs[steps-1][2]
        tc_list_c = []
        for row in obs:
            if row[1] == 1:
                if row[2] == tc or row[3] == tc: tc_list_c.append(row)
            elif row[0] == 1:
                if row[3] == tc: tc_list_c.append(row)
        tc = obs[steps-1][3]
        tc_list_t = []
        for row in obs:
            if row[1] == 1:
                if row[2] == tc or row[3] == tc: tc_list_t.append(row)
            elif row[0] == 1:
                if row[3] == tc: tc_list_t.append(row)
        if len(tc_list_c) > 1 and len(tc_list_t) > 1:
            if tc_list_c[-1] == tc_list_c[-2] and tc_list_t[-1] == tc_list_t[-2]: dup_r = -10

    ## 4. Gate Type Bias
    # Encourages the use of Parameterized Rotation gates (which provide trainability).
    if obs[steps-1][0] == 1:
        gate_r = 1
        rot_r = 1
    else:
        gate_r = 0
        rot_r = 0

    ## 5. Topology Constraint (CNOT Distance)
    # Rewards CNOT gates between physically adjacent or close qubits (Locality).
    if obs[steps-1][1] == 1:   
        cnot_r = 1 / abs(obs[steps-1][2]-obs[steps-1][3])
    else: cnot_r = 0    

    ## 6. Efficiency Reward (Circuit Depth)
    # Encourages shorter circuits by penalizing the number of steps.
    steps_r = (60 - steps) / 60
    
    ## Total Weighted Reward Calculation
    # Combines all components with specific hyperparameters.
    # Note: Accuracy has the highest weight (x30 effectively), prioritizing valid classification.
    return [acc_m, cost_m, gate_r, rot_r, cnot_r, steps_r, pop_r, dup_r], (acc_m - 0.5)*2 * 15 + cost_m * 2 + gate_r * 3 + rot_r + cnot_r + steps_r * 5 + pop_r * 3 + dup_r  ## with weight

In [ ]:
# Cell 15: Quantum Circuit Environment Class (OpenAI Gym-like Interface)
# Encapsulates the quantum circuit generation process as a Reinforcement Learning environment.
# Defines the Action Space, State Space (Observation), and Transition Logic.

class qc:
    """
    The environment class for the RL agent.
    Manages the state (current circuit structure), executes actions (adding gates),
    and calculates rewards based on the performance of the generated circuit.
    """
    def __init__(self):
        # Define Action Space: All possible gates the agent can select.
        # Includes parameterized rotations (Rot) on all axes (X,Y,Z) for all qubits,
        # and entangling gates (CNOT) for all possible pairs.
        self.gates = [['Rot','X', 0], ['Rot','X', 1], ['Rot','X', 2], ['Rot','X', 3], ['Rot','X', 4], ['Rot','X', 5],
         ['Rot','Y', 0], ['Rot','Y', 1], ['Rot','Y', 2], ['Rot','Y', 3], ['Rot','Y', 4], ['Rot','Y', 5],
         ['Rot','Z', 0], ['Rot','Z', 1], ['Rot','Z', 2], ['Rot','Z', 3], ['Rot','Z', 4], ['Rot','Z', 5],
         ['CNOT', 0, 1],  ['CNOT', 0, 2],  ['CNOT', 0, 3], ['CNOT', 0, 4], ['CNOT', 0, 5],
         ['CNOT', 1, 0],  ['CNOT', 1, 2],  ['CNOT', 1, 3], ['CNOT', 1, 4], ['CNOT', 1, 5],
         ['CNOT', 2, 0],  ['CNOT', 2, 1],  ['CNOT', 2, 3], ['CNOT', 2, 4], ['CNOT', 2, 5],
         ['CNOT', 3, 0],  ['CNOT', 3, 1],  ['CNOT', 3, 2], ['CNOT', 3, 4], ['CNOT', 3, 5],
         ['CNOT', 4, 0],  ['CNOT', 4, 1],  ['CNOT', 4, 2], ['CNOT', 4, 3], ['CNOT', 4, 5],
         ['CNOT', 5, 0],  ['CNOT', 5, 1],  ['CNOT', 5, 2], ['CNOT', 5, 3], ['CNOT', 5, 4]] 
        
        self.len_qc = 60 # Maximum circuit depth (time steps per episode)
        self.act_space = len(self.gates) # Size of the action space

    def reset(self):
        """
        Resets the environment to the initial state.
        Note: Unlike Model B, this method does not accept 'weights_ent' as input,
        confirming that no pre-trained entanglement layers are used. The agent starts with an empty circuit.
        """
        self.steps = 0
        self.obs = [[0] * 4 for _ in range(self.len_qc)] # Initialize empty observation grid
        self.gatestream = [] # Clear gate history
        self.reward = -1
        self.term = -1
        self.done = 0
        return

    def step(self, act):
        """
        Executes one time step within the environment.
        1. Updates the circuit structure with the chosen gate (action).
        2. Trains the current circuit parameters (weights_rl only) via 'opt_classifier'.
        3. Computes the reward based on accuracy, cost, and structural heuristics.
        """
        # Validity checks
        if act > self.act_space-1 or act < 0:
            print("out of action space")
            return 0
        if self.steps > self.len_qc-1:
            print("out of qc length")
            return 0
        
        # 1. State Transition
        # Apply the action to update the observation and gatestream.
        self.steps, self.obs, self.gatestream = update_obs(act, self.steps, self.obs, self.gatestream, self.gates)
        
        # 2. Performance Evaluation (Inner Loop Optimization)
        # Train the current circuit structure to determine its potential performance.
        # Note: Optimization targets only the RL-generated parameters.
        self.outs, self.draw, self.figset = opt_classifier(
            self.gatestream,
            iters=15, draw=False
        )
        
        # 3. Reward Calculation
        self.rlist, self.reward = cal_reward(self.steps, self.obs, self.outs)

        # 4. Termination Check (Max Steps)
        if self.steps == self.len_qc or self.done == 1:
            self.term = 1
        else: self.term = 0

        # 5. Success Check (Accuracy Threshold)
        # If the circuit achieves sufficient accuracy (>= 0.65), terminate early.
        acc_val_max   = max(row[3] for row in self.outs)
        if acc_val_max >= 0.65:  
            self.done = 1
            self.term = 1
        
        return 1

    def gs_step(self, gs):
        """
        Helper function to evaluate a specific gate sequence (gatestream) manually.
        """
        self.outs, self.draw, self.figset = opt_classifier(
            gs, iters=15, draw=False
        )
        return 1

    def sample(self):
        """
        Selects a random action from the action space (Exploration).
        """
        return random.randint(0, self.act_space-1)
    
    def showdb(self, figset, gatestream):
        """
        Visualizes the decision boundary of the trained model using PCA.
        Args:
            figset: Contains trained weights and dataset splits.
            gatestream: The circuit structure used for classification.
        """
        weights_rl, bias, X_train, Y_train, X_val, Y_val = figset
        
        X_all = torch.cat([X_train, X_val], dim=0)
        Y_all = torch.cat([Y_train, Y_val], dim=0)
        split = len(X_train)

        # Generate predictions using the Pure RL model (no weights_ent)
        preds = [torch.sign(variational_classifier(weights_rl, bias, x, gatestream)) for x in X_all]
        preds = torch.stack(preds).squeeze().cpu().numpy()

        # PCA Projection for 2D Visualization
        from sklearn.decomposition import PCA
        pca = PCA(n_components=2)
        X_2d = pca.fit_transform(X_all.cpu().numpy())

        # Determine Correct/Wrong predictions
        correct = (preds == Y_all.cpu().numpy())

        # Plotting
        plt.figure(figsize=(6,6))
        plt.scatter(X_2d[correct, 0], X_2d[correct, 1], c="g", label="Correct", alpha=0.7)
        plt.scatter(X_2d[~correct, 0], X_2d[~correct, 1], c="r", label="Wrong", alpha=0.7, marker="x")
    
        # Draw vertical line separating Train (left) and Val (right) implicitly by index if sorted, 
        # or just as a visual marker for the split point in the array.
        plt.axvline(X_2d[split, 0], color="gray", linestyle="--", alpha=0.3)

        plt.legend()
        plt.title("Classification results (PCA 2D projection)")
        plt.show()